# Demonstration of PyMC Markov Chain Monte Carlo  on a Nonlinear Model with 4 Parameters

In [ ]:
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import arviz as az
import seaborn as sns
import pandas as pd

# Configuration
np.random.seed(42)
param_names = ["m0", "m1", "m2", "m3"]
model_string = "y = m0 / (1 + exp(-m1*(x - m2))) + m3*sin(x)"
true_params = [3.0, 2.0, 5.0, 0.2]

def forward_model(x, m):
    return m[0] / (1 + np.exp(-m[1]*(x - m[2]))) + m[3] * np.sin(x)

# Simulated observations
sigma_low = 0.02
sigma_high = 2.0

x_data = np.linspace(0, 10, 800)
y_true = forward_model(x_data, true_params)
y_obs_low = y_true + np.random.normal(0, sigma_low, size=len(x_data))
y_obs_high = y_true + np.random.normal(0, sigma_high, size=len(x_data))

# pyMC parameters
target_accept = 0.9 
max_treedepth = 12
draws = 1000 
tune = 1000

In [ ]:
def run_pymc_sampling(x, y_obs, sigma, target_accept=0.8, max_treedepth=10, draws=1000, tune=1000):
    with pm.Model() as model:
        m0 = pm.Normal("m0", mu=1, sigma=2)
        m1 = pm.Normal("m1", mu=1, sigma=2)
        m2 = pm.Normal("m2", mu=5, sigma=2)
        m3 = pm.Normal("m3", mu=0, sigma=1)
        y_model = m0 / (1 + pm.math.exp(-m1*(x - m2))) + m3 * pm.math.sin(x)
        y_obs_ = pm.Normal("y_obs", mu=y_model, sigma=sigma, observed=y_obs)

        trace = pm.sample(draws=draws, tune=tune, target_accept=target_accept, max_treedepth=max_treedepth, return_inferencedata=True, random_seed=42)
    return trace

In [ ]:
def plot_data_model_fit(x, y_true, y_obs, mean_params, label):
    plt.figure(figsize=(10, 4))
    plt.plot(x, y_true, 'k-', label='True')
    plt.plot(x, y_obs, '.', label='Observed', alpha=0.4)
    plt.plot(x, forward_model(x, mean_params), 'r--', label='Inferred')
    plt.title(f'{label} Fit')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def triangle_plot(data, param_names, true_params, inferred_params, label, half_range=0.5):
    """
    Create a triangle plot showing posterior distributions.
    """
    if isinstance(data, np.ndarray):
        df = pd.DataFrame(data, columns=param_names)
    else:
        df = az.extract(data, var_names=param_names).to_dataframe()[param_names]
    
    means = df.mean()

    g = sns.PairGrid(df, vars=param_names, corner=True)
    g.map_lower(sns.kdeplot, levels=5, fill=False)
    g.map_diag(sns.kdeplot, fill=True)

    # Calculate dynamic axis limits
    lower_limits = []
    upper_limits = []
    for i, p in enumerate(param_names):
        mean_val = means[p]
        true_val = true_params[i]
        lower = min(mean_val, true_val) - half_range
        upper = max(mean_val, true_val) + half_range
        lower_limits.append(lower)
        upper_limits.append(upper)

    # Set axis limits for marginals and KDEs
    for i, p in enumerate(param_names):
        ax = g.axes[i, i]
        ax.axvline(true_params[i], color='k', linestyle='-')
        ax.axvline(inferred_params[i], color='r', linestyle='--')
        ax.set_xlim(lower_limits[i], upper_limits[i])

    for i in range(len(param_names)):
        for j in range(i):
            ax = g.axes[i, j]
            ax.set_xlim(lower_limits[j], upper_limits[j])
            ax.set_ylim(lower_limits[i], upper_limits[i])

    plt.suptitle(label + f" - Triangle Plot (half-range = {half_range})", y=1.02)
    plt.show()

def print_summary(model_string, true_vals, inferred_vals, label):
    print(f"--- {label} Summary ---")
    print(f"{model_string}")

    print(f"{'Param':<5} {'True':>10} {'Inferred':>10} {'Error':>10}")
    for name, t, p in zip(param_names, true_vals, inferred_vals):
        print(f"{name:<5} {t:10.4f} {p:10.4f} {p - t:10.4f}")

## 🔵 Inference on Low Noise Data

In [ ]:
sigma = sigma_low
label = f"Low Noise: sigma={sigma_low}"
y_obs = y_true + np.random.normal(0, sigma, size=len(x_data))

idata = run_pymc_sampling(x_data, y_obs, sigma, target_accept, max_treedepth, draws, tune)
mean = az.summary(idata, var_names=param_names)['mean'].values

print_summary(model_string, true_params, mean, label)
plot_data_model_fit(x_data, y_true, y_obs_low, mean, label)
az.plot_trace(idata, var_names=param_names, figsize=(10, 6), lines=[(k, {}, [v]) for k, v in zip(param_names, true_params)])
plt.suptitle(f"Trace Plots: {label} ", y=1.02)
plt.show()
triangle_plot(idata, param_names, true_params, mean, label, half_range=0.05)

## 🔴 Inference on High Noise Data

In [ ]:
sigma = sigma_high
label = f"High Noise: sigma={sigma_high}"
y_obs = y_true + np.random.normal(0, sigma, size=len(x_data))

idata = run_pymc_sampling(x_data, y_obs, sigma, target_accept, max_treedepth, draws, tune)
mean = az.summary(idata, var_names=param_names)['mean'].values

print_summary(model_string, true_params, mean, label)
plot_data_model_fit(x_data, y_true, y_obs_low, mean, label)
az.plot_trace(idata, var_names=param_names, figsize=(10, 6), lines=[(k, {}, [v]) for k, v in zip(param_names, true_params)])
plt.suptitle(f"Trace Plots: {label} ", y=1.02)
plt.show()
triangle_plot(idata, param_names, true_params, mean, label, half_range=0.5)